# Alignment Error Visualization

This notebook collects COM data from the database and tries to quantify some alignment errors. The main results are shown in the plots at the end of the notebook.

In [ ]:
import os
import sys
from pathlib import Path
import numpy as np
import SimpleITK as sitk
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt

PIPELINE_ROOT = Path('./').absolute().parents[1]
PIPELINE_ROOT = PIPELINE_ROOT.as_posix()
sys.path.append(PIPELINE_ROOT)
print(PIPELINE_ROOT)

In [ ]:
#from library.controller.sql_controller import SqlController
#from library.image_manipulation.filelocation_manager import FileLocationManager
from library.atlas.atlas_utilities import affine_transform_point, load_transformation
#from library.atlas.brain_structure_manager import BrainStructureManager
#from library.utilities.utilities_process import M_UM_SCALE, SCALING_FACTOR, random_string, \
#    read_image, write_image


In [ ]:
def get_origins(animal, scale):
    """
    Fetches the origins from disk. The data is already scaled to 10um.
    """

    origins = {}
    dirpath = f'/net/birdstore/Active_Atlas_Data/data_root/atlas_data/{animal}/origin'
    if not os.path.exists(dirpath):
        return origins
    files = sorted(os.listdir(dirpath))
    for file in files:
        structure = Path(file).stem
        filepath = os.path.join(dirpath, file)
        origin = np.loadtxt(filepath)
        origins[structure] = origin * scale
    return origins

def compute_affine_transformation(src_pts, dst_pts):
    """
    src_pts: Nx3 array of points in brain A
    dst_pts: Nx3 array of points in brain B
    
    Returns:
        T  : 4x4 affine matrix mapping src → dst
        err: RMS error after transform
    """

    src_pts = np.asarray(src_pts)
    dst_pts = np.asarray(dst_pts)
    assert src_pts.shape == dst_pts.shape
    assert src_pts.shape[0] >= 4  # minimum points for 3D affine

    N = src_pts.shape[0]

    # Build homogeneous design matrix: [x y z 1]
    X = np.hstack([src_pts, np.ones((N, 1))])        # Nx4
    Y = dst_pts                                       # Nx3

    # Solve X * A = Y  → A is 4×3 matrix
    A, *_ = np.linalg.lstsq(X, Y, rcond=None)

    # Convert to 4×4 transform
    T = np.eye(4)
    T[:3, :4] = A.T

    # Apply transform to src
    src_h = np.hstack([src_pts, np.ones((N, 1))])    # Nx4
    dst_pred = (T @ src_h.T).T[:, :3]

    # Compute RMS error
    rmse = np.sqrt(np.mean(np.sum((dst_pts - dst_pred)**2, axis=1)))

    return T, rmse, dst_pred


In [ ]:
moving_name = 'MD594'
fixed_name = 'Allen'
moving_all = get_origins(moving_name, scale=10)
fixed_all = get_origins(fixed_name, scale=10)
common_keys = list(moving_all.keys() & fixed_all.keys())
bad_keys = ('10N_L')
good_keys = set(common_keys) - set(bad_keys)
#good_keys = ['3N_L','3N_R', '4N_L','4N_R', '5N_L', '5N_R', '6N_L', '6N_R']
#good_keys.extend(['7N_L','7N_R', 'SC', 'IC'])

moving_src = np.array([moving_all[s] for s in good_keys])
fixed_src = np.array([fixed_all[s] for s in good_keys])
transformation_matrix, rmse, predictions = compute_affine_transformation(moving_src, fixed_src)
data_path = "/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration"
print(transformation_matrix)
print('# used keys', len(good_keys))

df_list = []
predicted_dict = {}
predictions = []
actuals = []
for structure in common_keys:    
    moving0 = np.array(moving_all[structure])
    fixed0 = np.array(fixed_all[structure]) 
    predicted0 = affine_transform_point(moving0, transformation_matrix)
    difference = [a - b for a, b in zip(moving0, fixed0)]
    abs_diff = sum([abs(a) for a in difference])
    row = [structure, np.round(moving0,2), np.round(fixed0,2), np.round(predicted0,2), abs_diff]
    df_list.append(row)
    predicted_dict[structure] = predicted0
    predictions.append(predicted0)
    actuals.append(fixed0)
cases = len(actuals)
print('len common keys', cases)
meanSquaredError = ((np.array(predictions) - np.array(actuals)) ** 2).mean() 
rmse = np.sqrt(meanSquaredError)
print('RMSE', rmse)

In [ ]:
# MD589 to Allen RMS 260.0211852431133
# MD585 to Allen RMS 263.314352291951
# MD594 to Allen RMS 250.79820210419254
# AtlasV8 DB to Allen RMS: 237.06805950085737 observations: 37
# MD585 to MD594 152.06606097021333 observations: 51
# MD585 to Allen 263.31435 observations: 37
# MD589 to Allen 260.02 observations: 37
# MD594 to Allen 250.79 observations: 37

In [ ]:
#transformation_matrix = np.hstack([transformation_matrix, t])
#transformation_matrix = np.vstack([transformation_matrix, np.array([0, 0, 0, 1])])
#print(transformation_matrix)
structure = 'SC'
try:
    com = moving_all[structure]
except KeyError:
    structure = common_keys[0]
    com = moving_all[structure]
#comtfm = np.array([824.6051918494063, 80.83004570523167, 363.4390121956811])
transformed_structure = affine_transform_point(com, transformation_matrix)

print(f'{moving_name} {structure} non trans {np.round(np.array(com))}')
print(f'{fixed_name} {structure} {np.round(np.array(fixed_all[structure]))}')
print(f'{moving_name} {structure} apply trans {np.round(transformed_structure,2)}')
diff = transformed_structure - fixed_all[structure]
#comdiff = comtfm - fixed_all[structure]

print(f'{moving_name}->{fixed_name} error {structure} {diff}')
#print(f'{moving_name}->{fixed_name} tfm error {structure} {comdiff}')

In [ ]:
columns = ['structure', moving_name, fixed_name, 'predicted', 'abs_diff']
df = pd.DataFrame(df_list, columns=columns)
df.index.name = 'Index'
df = df.round(2)
df.sort_values(by=['abs_diff'], inplace=True)
#df.to_csv('/home/eddyod/programming/pipeline/docs/sphinx/source/_static/results.csv', index=False)
df.head(50)
#20	3N_R	[1079.0, 531.0, 485.0]	[910.0, 380.0, 17.0]	[873.0, 358.0, 298.0]	[-37.0, -21.0, 281.0]	284.3851
#7	4N_L	[1135.0, 529.0, 423.0]	[959.0, 378.0, 544.0]	[923.0, 362.0, 356.0]	[-36.0, -16.0, -188.0]	192.0327

In [ ]:
def fit_affine_3d(src_pts, tgt_pts, return_residuals=False):
    """
    Compute best-fit affine transform (3D) mapping src_pts -> tgt_pts using least squares.

    Args:
        src_pts: (N,3) array-like of source fiducial points.
        tgt_pts: (N,3) array-like of target fiducial points.
        return_residuals: if True, also return the residual vector for each point (N,).

    Returns:
        H: (4,4) numpy array, homogeneous affine transform such that:
           transformed = (H @ [x,y,z,1])[:3] approximates target.
        rmse: float, root mean square error between transformed source and target points.
        (optional) residuals: (N,) array of Euclidean distances for each point (only if return_residuals=True)

    Notes:
        - For an exact unique solution you need >= 4 non-coplanar points; with fewer points
          the function still returns the least-squares best fit (may be underdetermined).
    """
    src = np.asarray(src_pts, dtype=float)
    tgt = np.asarray(tgt_pts, dtype=float)
    if src.shape != tgt.shape:
        raise ValueError("src_pts and tgt_pts must have the same shape (N,3).")
    if src.ndim != 2 or src.shape[1] != 3:
        raise ValueError("src_pts and tgt_pts must be (N,3) arrays.")

    N = src.shape[0]
    if N < 1:
        raise ValueError("At least one point is required.")

    # Build design matrix X: N x 4 with last column ones
    X = np.hstack([src, np.ones((N, 1), dtype=float)])   # shape (N,4)
    Y = tgt                                            # shape (N,3)

    # Solve for 3x4 matrix M^T (we solve X * M^T = Y so M^T = pinv(X) @ Y)
    # Using numpy.linalg.lstsq for numerical stability
    M_transpose, residuals_sum_sq, rank, s = np.linalg.lstsq(X, Y, rcond=None)
    # M_transpose shape: (4,3) because we solved X (N,4) * (4,3) = (N,3)
    M = M_transpose.T   # now (3,4)

    # Convert to homogeneous 4x4 matrix H
    H = np.eye(4, dtype=float)
    H[:3, :4] = M

    # Transform source pts
    src_h = np.hstack([src, np.ones((N, 1), dtype=float)])  # (N,4)
    transformed = (src_h @ M.T)  # (N,3)

    # Per-point Euclidean residuals and RMSE
    diffs = transformed - tgt
    dists = np.linalg.norm(diffs, axis=1)
    rmse = np.sqrt(np.mean(np.sum(diffs**2, axis=1)))

    if return_residuals:
        return H, rmse, dists
    else:
        return H, rmse


In [ ]:
R, error = fit_affine_3d(moving_src, fixed_src)
print(error)

In [ ]:
error